# 多层感知机
## 一、使用函数
* nn.Parameter():torch.nn.Parameter()将一个不可训练的tensor转换成可以训练的类型parameter，并将这个parameter绑定到这个module里面。即在定义网络时这个tensor就是一个可以训练的参数了。使用这个函数的目的也是想让某些变量在学习的过程中不断的修改其值以达到最优化。换句话说，在不使用神经网络层时它的作用是形成可更新的张量

## 二、从零开始实现

In [2]:
import torch
from torch import nn
import torchvision
from torch.utils import data
from torchvision import transforms
import matplotlib.pyplot as plt
# 返回批量batch_size的数据
def load_data_fashion_mnist(batch_size, resize=None):  #@save
    """下载Fashion-MNIST数据集，然后将其加载到内存中"""
    trans = [transforms.ToTensor()]# 一个函数的列表
    if resize:
        trans.insert(0, transforms.Resize(resize))# 函数功能增加一项
    trans = transforms.Compose(trans)# 组合图像操作
    # 与上面类似，下载图像集
    mnist_train = torchvision.datasets.FashionMNIST(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="../data", train=False, transform=trans, download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True,
                        ),# 使用几个进程进行加载，默认为0
            data.DataLoader(mnist_test, batch_size, shuffle=False,# 代表主线程加载
                            ))

e:\coding_setting\Miniconda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
batch_size=256
train_iter,test_iter=load_data_fashion_mnist(batch_size)

> 推导过程：
> Fashion-MNIST中的每个图像由$28*28=784$个灰度像素值组成。所有图像共分为10个类别，也就是说输入特征数为784，最后的输出特征为10，也就是说输入层为$x \in R^{256\times784}$, 隐藏层的隐藏个数为256个，也就是说明$w1 \in R^{784\times256}$$b1 \in R^{1\times256}$
> 输出层为10个特征，则$w2 \in R^{256\times10}$,$b2 \in R^{1\times10}$
> 

In [4]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256

W1 = nn.Parameter(torch.randn(
    num_inputs, num_hiddens, requires_grad=True) * 0.01)
b1 = nn.Parameter(torch.zeros(num_hiddens, requires_grad=True))
W2 = nn.Parameter(torch.randn(
    num_hiddens, num_outputs, requires_grad=True) * 0.01)
b2 = nn.Parameter(torch.zeros(num_outputs, requires_grad=True))

params = [W1, b1, W2, b2]

In [5]:
# 只保留正数的激活函数
def relu(X):
    a = torch.zeros_like(X)
    return torch.max(X, a)

In [6]:
def net(X):
    X = X.reshape((-1, num_inputs))
    H = relu(X@W1 + b1)  # 这里“@”代表矩阵乘法
    return (H@W2 + b2)

In [7]:
#使用softmax实现中使用的softmax函数
loss = nn.CrossEntropyLoss(reduction='none')

In [8]:
def accuracy(y_hat, y):  #@save
    """计算预测正确率"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())/y.shape[0]

In [9]:
def train(net,num_epochs,train_iter,test_iter,loss,trainer):
    all_accuracy=0.0
    for i in range(num_epochs):
        for X,y in train_iter:
            y_hat=net(X)
            l=loss(y_hat,y)
            trainer.zero_grad()
            l.mean().backward()
            trainer.step()
        one_accuracy=accuracy(y_hat,y)
        all_accuracy+=one_accuracy
        print(f'第{i+1}次测试,损失率为{l.mean()},准确率为{one_accuracy}')
    print(f'平均准确率为{all_accuracy/num_epochs}')

In [10]:
num_epochs, lr = 10, 0.1
updater = torch.optim.SGD(params, lr=lr)
train(net, num_epochs,train_iter, test_iter, loss, updater)

第1次测试,损失率为0.5537840723991394,准确率为0.7916666666666666
第2次测试,损失率为0.49689531326293945,准确率为0.8229166666666666
第3次测试,损失率为0.3481275141239166,准确率为0.9270833333333334
第4次测试,损失率为0.3859090507030487,准确率为0.8645833333333334
第5次测试,损失率为0.44069480895996094,准确率为0.84375
第6次测试,损失率为0.36168432235717773,准确率为0.8958333333333334
第7次测试,损失率为0.4410228729248047,准确率为0.84375
第8次测试,损失率为0.3754465579986572,准确率为0.8645833333333334
第9次测试,损失率为0.3595719337463379,准确率为0.8854166666666666
第10次测试,损失率为0.35955002903938293,准确率为0.90625
平均准确率为0.8645833333333333


## 三、使用深度学习框架

In [15]:
net = nn.Sequential(nn.Flatten(),
                    nn.Linear(784, 256),
                    nn.Tanh(),
                    nn.Linear(256, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

In [16]:
batch_size, lr, num_epochs = 256, 0.1, 10
loss = nn.CrossEntropyLoss(reduction='none')
trainer = torch.optim.SGD(net.parameters(), lr=lr)

train_iter, test_iter = load_data_fashion_mnist(batch_size)
train(net, num_epochs,train_iter, test_iter, loss,  trainer)

第1次测试,损失率为0.5564848184585571,准确率为0.8020833333333334
第2次测试,损失率为0.5153186321258545,准确率为0.8645833333333334
第3次测试,损失率为0.483040452003479,准确率为0.7916666666666666
第4次测试,损失率为0.4980808198451996,准确率为0.8333333333333334
第5次测试,损失率为0.5532759428024292,准确率为0.8020833333333334
第6次测试,损失率为0.45694032311439514,准确率为0.84375
第7次测试,损失率为0.607175886631012,准确率为0.84375
第8次测试,损失率为0.4522967040538788,准确率为0.8229166666666666
第9次测试,损失率为0.40743720531463623,准确率为0.8229166666666666
第10次测试,损失率为0.3081527650356293,准确率为0.8854166666666666
平均准确率为0.83125


## 四、总结
* 多层感知机的核心就是采用一层或多层的隐藏层执行非线性仿射变化，来消除单一线性变化带来的误差